In [2]:
from datetime import datetime


with open('/codilime/data/train.csv') as train_file:
    content = train_file.readlines()
    content = [x.strip() for x in content]
    data = []
    for line in content:
        mylist = line.split(',')
        data.append(mylist)

print(data[:10])

[['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count'], ['2011-01-01 00:00:00', '1', '0', '0', '1', '9.84', '14.395', '81', '0', '3', '13', '16'], ['2011-01-01 01:00:00', '1', '0', '0', '1', '9.02', '13.635', '80', '0', '8', '32', '40'], ['2011-01-01 02:00:00', '1', '0', '0', '1', '9.02', '13.635', '80', '0', '5', '27', '32'], ['2011-01-01 03:00:00', '1', '0', '0', '1', '9.84', '14.395', '75', '0', '3', '10', '13'], ['2011-01-01 04:00:00', '1', '0', '0', '1', '9.84', '14.395', '75', '0', '0', '1', '1'], ['2011-01-01 05:00:00', '1', '0', '0', '2', '9.84', '12.88', '75', '6.0032', '0', '1', '1'], ['2011-01-01 06:00:00', '1', '0', '0', '1', '9.02', '13.635', '80', '0', '2', '0', '2'], ['2011-01-01 07:00:00', '1', '0', '0', '1', '8.2', '12.88', '86', '0', '1', '2', '3'], ['2011-01-01 08:00:00', '1', '0', '0', '1', '9.84', '14.395', '75', '0', '1', '7', '8']]


# Check hypothesis

1. Less people use when rainy weather?

In [1]:
import xgboost as xgb

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Initial data preprocessing

Notes: Date is not important. Only hour is important.

In [3]:
def format_date_to_hour(dataset):
    dataset_processed = []
    for row in dataset[1:]:
        processed_row = []
        row_date_hour = datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S').hour
        processed_row.append(row_date_hour)
        
        for element in row[1:]:
            processed_row.append(float(element))
            
        dataset_processed.append(processed_row)
    
    return dataset_processed
            
    
data_processed = format_date_to_hour(data)

In [4]:
data_processed

[[0, 1.0, 0.0, 0.0, 1.0, 9.84, 14.395, 81.0, 0.0, 3.0, 13.0, 16.0],
 [1, 1.0, 0.0, 0.0, 1.0, 9.02, 13.635, 80.0, 0.0, 8.0, 32.0, 40.0],
 [2, 1.0, 0.0, 0.0, 1.0, 9.02, 13.635, 80.0, 0.0, 5.0, 27.0, 32.0],
 [3, 1.0, 0.0, 0.0, 1.0, 9.84, 14.395, 75.0, 0.0, 3.0, 10.0, 13.0],
 [4, 1.0, 0.0, 0.0, 1.0, 9.84, 14.395, 75.0, 0.0, 0.0, 1.0, 1.0],
 [5, 1.0, 0.0, 0.0, 2.0, 9.84, 12.88, 75.0, 6.0032, 0.0, 1.0, 1.0],
 [6, 1.0, 0.0, 0.0, 1.0, 9.02, 13.635, 80.0, 0.0, 2.0, 0.0, 2.0],
 [7, 1.0, 0.0, 0.0, 1.0, 8.2, 12.88, 86.0, 0.0, 1.0, 2.0, 3.0],
 [8, 1.0, 0.0, 0.0, 1.0, 9.84, 14.395, 75.0, 0.0, 1.0, 7.0, 8.0],
 [9, 1.0, 0.0, 0.0, 1.0, 13.12, 17.425, 76.0, 0.0, 8.0, 6.0, 14.0],
 [10, 1.0, 0.0, 0.0, 1.0, 15.58, 19.695, 76.0, 16.9979, 12.0, 24.0, 36.0],
 [11, 1.0, 0.0, 0.0, 1.0, 14.76, 16.665, 81.0, 19.0012, 26.0, 30.0, 56.0],
 [12, 1.0, 0.0, 0.0, 1.0, 17.22, 21.21, 77.0, 19.0012, 29.0, 55.0, 84.0],
 [13, 1.0, 0.0, 0.0, 2.0, 18.86, 22.725, 72.0, 19.9995, 47.0, 47.0, 94.0],
 [14, 1.0, 0.0, 0.0, 2.0, 18.86